# **Download** the concatenated synthetic-profile file
e.g., https://data-argo.ifremer.fr/dac/coriolis/6903574/6903574_Sprof.nc

### <span style="color:red">--- User input begins ---</span>
- Specify the WMOID(s) of the BGC-Argo float(s) you want to download
- You can specify more than one float as a list if you wish

In [1]:
wmoids = [6903574]

### <span style="color:red">--- User input ends ---</span>

### Download

In [2]:
import pandas as pd
import os
import urllib.request

# Sometimes either the French or the US server is down. So try both HTTPS 
urls_to_try = [
    'https://data-argo.ifremer.fr/',
    'https://usgodae.org/pub/outgoing/argo/'
]

# 2. Find a working server and load the index once
index_df = None
active_url = None

for base_url in urls_to_try:
    try:
        # We only need 'file' column, which saves memory
        index_df = pd.read_csv(base_url + "argo_synthetic-profile_index.txt", 
                               skiprows=8, usecols=['file'])
        active_url = base_url
        print(f"Successfully connected to {active_url}")
        break
    except Exception as e:
        print(f"{base_url} is not working...")

# 3. If a server is working, loop through each WMO ID
if index_df is not None:
    for wmoid in wmoids:
        wmo_str = str(wmoid)
        os.makedirs(wmo_str, exist_ok=True)
        
        # Look for the float in the index
        match = index_df[index_df['file'].str.contains(wmo_str)]
        
        if not match.empty:
            # Extract DAC name (e.g., 'aoml' or 'coriolis')
            file_path = match['file'].values[0]
            orgname = file_path.split('/')[0]
            
            # Construct the Sprof URL
            file_name = f"{wmo_str}_Sprof.nc"
            url = f"{active_url}dac/{orgname}/{wmo_str}/{file_name}"
            destination = f"{wmo_str}/{file_name}"
            
            try:
                urllib.request.urlretrieve(url, destination)
                print(f"Download successful: {destination}")
            except:
                print(f"Failed to download {wmo_str} from {url}")
        else:
            print(f"WMO ID {wmo_str} not found in the index file.")
else:
    print("Could not connect to any Argo servers. Please check your internet.")

Successfully read the index file from https://data-argo.ifremer.fr/
Download successful: 6903574/6903574_Sprof.nc
